In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/task1-sub1b/sample_submission.csv
/kaggle/input/task1-sub1b/news_train.csv
/kaggle/input/task1-sub1b/test.csv


In [62]:
#Importing all the libraries needed for the processes ahead
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud ,STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize 



In [ ]:
#Warnings removed
import re
import warnings
warnings.filterwarnings("ignore")

In [63]:
#Reading dataset
data = pd.read_csv("/kaggle/input/task1-sub1b/news_train.csv")
test_data = pd.read_csv("/kaggle/input/task1-sub1b/test.csv")


In [64]:
data.head()

,ID,News_title,News_headline,Category
0,1,"Do men enjoy sex more, or women? The Mahabhara...","[Book Extract] From Anushasana Parva, translat...",Arts
1,2,Why you should eat the Demonetisation laddoo,One laddoo equals to one lakh in your Jan Dhan...,humour
2,3,Is the world headed for a new Cold War?,The battle lines have become very clear with R...,politics
3,4,"Demonetisation is all about Modi, either you'r...",How many times should this hypocritical drama ...,business
4,5,Why electoral bonds won't clean up political f...,Union finance minister Arun Jaitley needs to p...,politics


In [65]:
test_data.head()

,ID,News_title,News_headline
0,1,How Blockchain can usher land reforms in the c...,"With multiple entities in the system, entruste..."
1,2,What Kamal Nath as president means for Congres...,With anti-incumbency building up against the S...
2,3,Why BJP's sweep in Rajasthan civic polls shoul...,The party lost in the chief minister's and her...
3,4,Harsha Bhogle got the boot for rising intolera...,It is Indian cricket's loss for it has once ag...
4,5,Amit Shah fighting Lok Sabha in Gandhinagar is...,Amit Shah has proven his capabilities often. T...


In [66]:
data['Category'].unique()


array(['Arts', 'humour', 'politics', 'business', 'sports', 'tech'],
      dtype=object)

In [67]:
data.shape


(15576, 4)

In [68]:
test_data.shape


(3484, 3)

In [69]:
data.dtypes


ID                int64
News_title       object
News_headline    object
Category         object
dtype: object

In [70]:
data.isnull().any()


ID               False
News_title       False
News_headline     True
Category         False
dtype: bool

In [71]:
test_data.isnull().any()

ID               False
News_title       False
News_headline     True
dtype: bool

In [72]:
df = data.iloc[:15576]
df


,ID,News_title,News_headline,Category
0,1,"Do men enjoy sex more, or women? The Mahabhara...","[Book Extract] From Anushasana Parva, translat...",Arts
1,2,Why you should eat the Demonetisation laddoo,One laddoo equals to one lakh in your Jan Dhan...,humour
2,3,Is the world headed for a new Cold War?,The battle lines have become very clear with R...,politics
3,4,"Demonetisation is all about Modi, either you'r...",How many times should this hypocritical drama ...,business
4,5,Why electoral bonds won't clean up political f...,Union finance minister Arun Jaitley needs to p...,politics
...,...,...,...,...
15571,15572,[Watch] Hansal Mehta beautifully captures Rohi...,'The Last Letter: Reach for the Stars' was sho...,politics
15572,15573,How Pakistan weakened Taliban by revealing Mul...,This is essential for getting the support of t...,politics
15573,15574,2G 'scam': What the verdict means for DMK,The real story will be in how DMK working pres...,politics
15574,15575,Kargil to Pathankot: Why is India never prepar...,It appears that history is constantly repeatin...,politics


In [73]:
print(type(test_data))

<class 'pandas.core.frame.DataFrame'>


In [74]:
t_df = test_data.iloc[:3484]
t_df


,ID,News_title,News_headline
0,1,How Blockchain can usher land reforms in the c...,"With multiple entities in the system, entruste..."
1,2,What Kamal Nath as president means for Congres...,With anti-incumbency building up against the S...
2,3,Why BJP's sweep in Rajasthan civic polls shoul...,The party lost in the chief minister's and her...
3,4,Harsha Bhogle got the boot for rising intolera...,It is Indian cricket's loss for it has once ag...
4,5,Amit Shah fighting Lok Sabha in Gandhinagar is...,Amit Shah has proven his capabilities often. T...
...,...,...,...
3479,3480,What game are BJP and Opposition playing over ...,This time the Congress plans to announce the c...
3480,3481,Afghanistan represents victory of India's soft...,PM Modi's visit to Kabul may help rejuvenate t...
3481,3482,Why warring AIADMK factions want a piece of Ja...,Move will also ensure Sasikala cannot try to g...
3482,3483,Mahasweta Devi was mother to downtrodden and S...,Her editorials against police excesses in Nand...


In [75]:
print(type(t_df))

<class 'pandas.core.frame.DataFrame'>


We try to Combine both the Title and Headline columns and then apply the preprocessing techniques on the single column.

In [76]:
df['News'] = df['News_title'] + " " + df['News_headline']
df


,ID,News_title,News_headline,Category,News
0,1,"Do men enjoy sex more, or women? The Mahabhara...","[Book Extract] From Anushasana Parva, translat...",Arts,"Do men enjoy sex more, or women? The Mahabhara..."
1,2,Why you should eat the Demonetisation laddoo,One laddoo equals to one lakh in your Jan Dhan...,humour,Why you should eat the Demonetisation laddoo O...
2,3,Is the world headed for a new Cold War?,The battle lines have become very clear with R...,politics,Is the world headed for a new Cold War? The ba...
3,4,"Demonetisation is all about Modi, either you'r...",How many times should this hypocritical drama ...,business,"Demonetisation is all about Modi, either you'r..."
4,5,Why electoral bonds won't clean up political f...,Union finance minister Arun Jaitley needs to p...,politics,Why electoral bonds won't clean up political f...
...,...,...,...,...,...
15571,15572,[Watch] Hansal Mehta beautifully captures Rohi...,'The Last Letter: Reach for the Stars' was sho...,politics,[Watch] Hansal Mehta beautifully captures Rohi...
15572,15573,How Pakistan weakened Taliban by revealing Mul...,This is essential for getting the support of t...,politics,How Pakistan weakened Taliban by revealing Mul...
15573,15574,2G 'scam': What the verdict means for DMK,The real story will be in how DMK working pres...,politics,2G 'scam': What the verdict means for DMK The ...
15574,15575,Kargil to Pathankot: Why is India never prepar...,It appears that history is constantly repeatin...,politics,Kargil to Pathankot: Why is India never prepar...


In [77]:
t_df['News'] = t_df['News_title'] + " " + t_df['News_headline']
t_df

,ID,News_title,News_headline,News
0,1,How Blockchain can usher land reforms in the c...,"With multiple entities in the system, entruste...",How Blockchain can usher land reforms in the c...
1,2,What Kamal Nath as president means for Congres...,With anti-incumbency building up against the S...,What Kamal Nath as president means for Congres...
2,3,Why BJP's sweep in Rajasthan civic polls shoul...,The party lost in the chief minister's and her...,Why BJP's sweep in Rajasthan civic polls shoul...
3,4,Harsha Bhogle got the boot for rising intolera...,It is Indian cricket's loss for it has once ag...,Harsha Bhogle got the boot for rising intolera...
4,5,Amit Shah fighting Lok Sabha in Gandhinagar is...,Amit Shah has proven his capabilities often. T...,Amit Shah fighting Lok Sabha in Gandhinagar is...
...,...,...,...,...
3479,3480,What game are BJP and Opposition playing over ...,This time the Congress plans to announce the c...,What game are BJP and Opposition playing over ...
3480,3481,Afghanistan represents victory of India's soft...,PM Modi's visit to Kabul may help rejuvenate t...,Afghanistan represents victory of India's soft...
3481,3482,Why warring AIADMK factions want a piece of Ja...,Move will also ensure Sasikala cannot try to g...,Why warring AIADMK factions want a piece of Ja...
3482,3483,Mahasweta Devi was mother to downtrodden and S...,Her editorials against police excesses in Nand...,Mahasweta Devi was mother to downtrodden and S...


In [78]:
print(type(t_df))

<class 'pandas.core.frame.DataFrame'>


In [79]:
df = df.drop(['News_headline', 'News_title'], axis=1)
df


,ID,Category,News
0,1,Arts,"Do men enjoy sex more, or women? The Mahabhara..."
1,2,humour,Why you should eat the Demonetisation laddoo O...
2,3,politics,Is the world headed for a new Cold War? The ba...
3,4,business,"Demonetisation is all about Modi, either you'r..."
4,5,politics,Why electoral bonds won't clean up political f...
...,...,...,...
15571,15572,politics,[Watch] Hansal Mehta beautifully captures Rohi...
15572,15573,politics,How Pakistan weakened Taliban by revealing Mul...
15573,15574,politics,2G 'scam': What the verdict means for DMK The ...
15574,15575,politics,Kargil to Pathankot: Why is India never prepar...


In [80]:
t_df = t_df.drop(['News_headline', 'News_title'], axis=1)
t_df

,ID,News
0,1,How Blockchain can usher land reforms in the c...
1,2,What Kamal Nath as president means for Congres...
2,3,Why BJP's sweep in Rajasthan civic polls shoul...
3,4,Harsha Bhogle got the boot for rising intolera...
4,5,Amit Shah fighting Lok Sabha in Gandhinagar is...
...,...,...
3479,3480,What game are BJP and Opposition playing over ...
3480,3481,Afghanistan represents victory of India's soft...
3481,3482,Why warring AIADMK factions want a piece of Ja...
3482,3483,Mahasweta Devi was mother to downtrodden and S...


In [81]:
print(type(t_df))


<class 'pandas.core.frame.DataFrame'>


In [82]:
cell_val = df.iloc[1,2]
cell_val

'Why you should eat the Demonetisation laddoo One laddoo equals to one lakh in your Jan Dhan account by next election.'

In [83]:
df.isnull().sum()


ID           0
Category     0
News        76
dtype: int64

In [84]:
t_df.isnull().sum()

ID       0
News    16
dtype: int64

In [85]:
print(type(t_df))

<class 'pandas.core.frame.DataFrame'>


In [86]:
df = df.dropna(subset=['News'])
df


,ID,Category,News
0,1,Arts,"Do men enjoy sex more, or women? The Mahabhara..."
1,2,humour,Why you should eat the Demonetisation laddoo O...
2,3,politics,Is the world headed for a new Cold War? The ba...
3,4,business,"Demonetisation is all about Modi, either you'r..."
4,5,politics,Why electoral bonds won't clean up political f...
...,...,...,...
15571,15572,politics,[Watch] Hansal Mehta beautifully captures Rohi...
15572,15573,politics,How Pakistan weakened Taliban by revealing Mul...
15573,15574,politics,2G 'scam': What the verdict means for DMK The ...
15574,15575,politics,Kargil to Pathankot: Why is India never prepar...


In [87]:
print(type(t_df))

<class 'pandas.core.frame.DataFrame'>


In [88]:
import numpy as np

# Ensure correct reference to the 'News' column
non_missing_values = t_df['News'].dropna().values  # Dropping missing values from 'News'
t_df['News'].fillna(np.random.choice(non_missing_values), inplace=True)  # Filling NaN with random values from non-missing entries

In [89]:
df = df.iloc[:15500]

In [90]:
df

,ID,Category,News
0,1,Arts,"Do men enjoy sex more, or women? The Mahabhara..."
1,2,humour,Why you should eat the Demonetisation laddoo O...
2,3,politics,Is the world headed for a new Cold War? The ba...
3,4,business,"Demonetisation is all about Modi, either you'r..."
4,5,politics,Why electoral bonds won't clean up political f...
...,...,...,...
15571,15572,politics,[Watch] Hansal Mehta beautifully captures Rohi...
15572,15573,politics,How Pakistan weakened Taliban by revealing Mul...
15573,15574,politics,2G 'scam': What the verdict means for DMK The ...
15574,15575,politics,Kargil to Pathankot: Why is India never prepar...


In [91]:
t_df

,ID,News
0,1,How Blockchain can usher land reforms in the c...
1,2,What Kamal Nath as president means for Congres...
2,3,Why BJP's sweep in Rajasthan civic polls shoul...
3,4,Harsha Bhogle got the boot for rising intolera...
4,5,Amit Shah fighting Lok Sabha in Gandhinagar is...
...,...,...
3479,3480,What game are BJP and Opposition playing over ...
3480,3481,Afghanistan represents victory of India's soft...
3481,3482,Why warring AIADMK factions want a piece of Ja...
3482,3483,Mahasweta Devi was mother to downtrodden and S...


In [92]:
print(type(t_df))

<class 'pandas.core.frame.DataFrame'>


In [93]:
t_df.isnull().sum()

ID      0
News    0
dtype: int64

In [94]:
t_df.duplicated().sum()


0

In [95]:
#This function is designed to remove HTML or XML tags from a given string using regular expressions
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [96]:
df['News'] = df['News'].apply(remove_tags)
df


,ID,Category,News
0,1,Arts,"Do men enjoy sex more, or women? The Mahabhara..."
1,2,humour,Why you should eat the Demonetisation laddoo O...
2,3,politics,Is the world headed for a new Cold War? The ba...
3,4,business,"Demonetisation is all about Modi, either you'r..."
4,5,politics,Why electoral bonds won't clean up political f...
...,...,...,...
15571,15572,politics,[Watch] Hansal Mehta beautifully captures Rohi...
15572,15573,politics,How Pakistan weakened Taliban by revealing Mul...
15573,15574,politics,2G 'scam': What the verdict means for DMK The ...
15574,15575,politics,Kargil to Pathankot: Why is India never prepar...


In [97]:
t_df['News'] = t_df['News'].apply(remove_tags)
t_df

,ID,News
0,1,How Blockchain can usher land reforms in the c...
1,2,What Kamal Nath as president means for Congres...
2,3,Why BJP's sweep in Rajasthan civic polls shoul...
3,4,Harsha Bhogle got the boot for rising intolera...
4,5,Amit Shah fighting Lok Sabha in Gandhinagar is...
...,...,...
3479,3480,What game are BJP and Opposition playing over ...
3480,3481,Afghanistan represents victory of India's soft...
3481,3482,Why warring AIADMK factions want a piece of Ja...
3482,3483,Mahasweta Devi was mother to downtrodden and S...


In [98]:
print(type(t_df))

<class 'pandas.core.frame.DataFrame'>


We try to convert all words into common lowercase and then try removing and filtering out all the stopwords.

In [99]:
df['News'] = df['News'].apply(lambda x:x.lower())
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['News'] = df['News'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [100]:
df

,ID,Category,News
0,1,Arts,"men enjoy sex more, women? mahabharata answers..."
1,2,humour,eat demonetisation laddoo one laddoo equals on...
2,3,politics,world headed new cold war? battle lines become...
3,4,business,"demonetisation modi, either india many times h..."
4,5,politics,electoral bonds clean political funding union ...
...,...,...,...
15571,15572,politics,[watch] hansal mehta beautifully captures rohi...
15572,15573,politics,pakistan weakened taliban revealing mullah oma...
15573,15574,politics,2g 'scam': verdict means dmk real story dmk wo...
15574,15575,politics,kargil pathankot: india never prepared terror ...


In [101]:
t_df['News'] = t_df['News'].apply(remove_tags)
t_df

,ID,News
0,1,How Blockchain can usher land reforms in the c...
1,2,What Kamal Nath as president means for Congres...
2,3,Why BJP's sweep in Rajasthan civic polls shoul...
3,4,Harsha Bhogle got the boot for rising intolera...
4,5,Amit Shah fighting Lok Sabha in Gandhinagar is...
...,...,...
3479,3480,What game are BJP and Opposition playing over ...
3480,3481,Afghanistan represents victory of India's soft...
3481,3482,Why warring AIADMK factions want a piece of Ja...
3482,3483,Mahasweta Devi was mother to downtrodden and S...


In [102]:
cell_value = df.iloc[1,2]
cell_value

'eat demonetisation laddoo one laddoo equals one lakh jan dhan account next election.'

In [103]:
print(type(t_df))

<class 'pandas.core.frame.DataFrame'>


In [104]:
cell_value = t_df.iloc[1,1]
cell_value

'What Kamal Nath as president means for Congress in Madhya Pradesh With anti-incumbency building up against the Shivraj Singh Chouhan, hopes for Congress have found a new life.'

In [105]:

y = df['Category']

In [106]:
x = df.iloc[:, [0, 2]]
x


,ID,News
0,1,"men enjoy sex more, women? mahabharata answers..."
1,2,eat demonetisation laddoo one laddoo equals on...
2,3,world headed new cold war? battle lines become...
3,4,"demonetisation modi, either india many times h..."
4,5,electoral bonds clean political funding union ...
...,...,...
15571,15572,[watch] hansal mehta beautifully captures rohi...
15572,15573,pakistan weakened taliban revealing mullah oma...
15573,15574,2g 'scam': verdict means dmk real story dmk wo...
15574,15575,kargil pathankot: india never prepared terror ...


In [107]:
y

0            Arts
1          humour
2        politics
3        business
4        politics
           ...   
15571    politics
15572    politics
15573    politics
15574    politics
15575    politics
Name: Category, Length: 15500, dtype: object

LabelEncoder is used to encode every Category into Numbers as:
Arts - 0
Business -1
Humour - 2
Politics - 3
Sports - 4
Tech - 5


In [108]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)
y

array([0, 2, 3, ..., 3, 3, 3])

In [109]:

print(type(t_df))

<class 'pandas.core.frame.DataFrame'>


In [110]:
print(type(t_df))        # Should print: <class 'pandas.core.frame.DataFrame'>


<class 'pandas.core.frame.DataFrame'>


In [132]:
x['News'] = x['News'].str.lower()
t_df['News'] = t_df['News'].str.lower()

In [164]:
import nltk

In [170]:
nltk.download('punkt', download_dir = '/root/nltk_data')
nltk.download('wordnet', download_dir = '/root/nltk_data')
nltk.download('omw-1.4', download_dir = '/root/nltk_data')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [171]:
print(nltk.data.path)

['/root/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/usr/share/nltk_data']


In [173]:
from nltk.corpus import wordnet

In [152]:
#Bag-of-words with the following features
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, min_df=2, max_df=0.8)

# Fit the vectorizer on the training data and transform it
x_train_bow = cv.fit_transform(df['News']).toarray()

# Transform the preprocessed test data using the same vectorizer
x_test = cv.transform(t_df['News']).toarray()



In [153]:
x_test.shape



(3484, 1500)

In [154]:
print(x_train_bow.shape)  # Should print something like (12400, n_features)
print(len(y))


(15500, 1500)
15500


In [155]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Fit the XGBoost model on the training data
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.3,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

xgb_model.fit(x_train_bow, y)

# Predict on the test data
y_pred = xgb_model.predict(x_test)

In [156]:
y_pred


array([3, 3, 3, ..., 3, 3, 3])

In [157]:
id_list = []
for i in range(len(y_pred)):
    id_list.append(i+1)

In [158]:
pred_df = pd.DataFrame({
    'ID': id_list,
    'Category': y_pred
})

In [159]:
pred_df.shape

(3484, 2)

In [ ]:
#Submit as csv
pred_df.to_csv('231EC106_Task1b.csv',index=False)